In [1]:
from selenium import webdriver
from selenium.common import WebDriverException
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
import undetected_chromedriver as uc
import logging
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime
import pandas as pd

In [2]:
job_cards_XPATH = '//div[contains(@class, "JobCard_jobCardWrapper")]'
jd_showMore_CSS = ".ShowMoreCTA_showMore__EtZpZ.ShowMoreCTA_spacing-md__bS21L"
company_name_CSS = ".heading_Heading__BqX5J.heading_Subhead__Ip1aW"
company_name_Xpath_alt = "/html[1]/body[1]/div[1]/div[4]/div[4]/div[2]/div[2]/div[1]/div[1]/header[1]/div[1]/div[1]/a[1]/div[1]/div[1]/div[1]/h4[1]"
rating_XPATH = "/html/body/div[5]/div[4]/div[2]/div[2]/div/div[1]/header/div[1]/div[1]/a/div[2]/div/div[2]/span"
job_title_class = "heading_Level1__soLZs"
location_css = "div[data-test='location']"
salary_class = "SalaryEstimate_salaryRange__brHFy"
jd_class = "Section_sectionComponent__nRsB2"

In [3]:
def get_jobs(keyword, num_jobs, sleep):

    options = uc.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = uc.Chrome(options=options)
    driver.maximize_window()
    keyword_redo = keyword.lower().strip().replace(" ", "-")
    url = f"https://www.glassdoor.com/Job/united-states-{keyword_redo}-jobs-SRCH_IL.0,13_IN1_KO14,{len(keyword_redo)+14}.htm?fromAge=14"
    driver.get(url)
    time.sleep(sleep)

    jobs = []
    start_index = 0

    try:
        while len(jobs) < num_jobs:
            job_cards = driver.find_elements(By.XPATH, '//div[contains(@class, "JobCard_jobCardWrapper")]')

            for i in range(start_index, len(job_cards)):
                print(f'Progress: {len(jobs)} / {num_jobs}')
                try:
                    driver.find_element(By.CLASS_NAME, "CloseButton").click()
                except:
                    pass

                job = job_cards[i]
                driver.execute_script("arguments[0].scrollIntoView(true);", job)
                job.click()

                try:
                    WebDriverWait(driver, 3).until(EC.presence_of_element_located((
                        By.CLASS_NAME, salary_class)))
                except:
                    pass
   
                try:
                    WebDriverWait(driver, 7).until(EC.presence_of_element_located((
                        By.CSS_SELECTOR, jd_showMore_CSS)))
                except:
                    pass

                try:
                    driver.find_element(By.CSS_SELECTOR, jd_showMore_CSS).click()
                    time.sleep(1)
                except:
                    pass

                collected = False
                while not collected:
                    try:
                        try:
                            Company_Name = driver.find_element(By.CSS_SELECTOR, company_name_CSS).text
                        except:
                            try:
                                Company_Name = driver.find_element(By.XPATH, company_name_Xpath_alt).text
                            except:
                                Company_Name = -1

                        try:
                            Rating = driver.find_element(By.XPATH, rating_XPATH).text
                        except:
                            Rating = -1

                        try:
                            Job_Title = driver.find_element(By.CLASS_NAME, job_title_class).text
                        except:
                            Job_Title = -1

                        try:
                            Location = driver.find_element(By.CSS_SELECTOR, location_css).text
                        except:
                            Location = -1

                        try:
                            Salary_Estimate = driver.find_element(By.CLASS_NAME, salary_class).text
                        except:
                            Salary_Estimate = -1

                        try:
                            Job_Description = driver.find_element(By.CLASS_NAME, jd_class).text
                        except:
                            Job_Description = -1

                        jobs.append({
                            "Company Name": Company_Name,
                            "Job Title": Job_Title,
                            "Location": Location,
                            "Job Description": Job_Description,
                            "Salary Estimate": Salary_Estimate,
                            "Scrapped_date": datetime.now().strftime("%m-%d-%Y")
                        })

                        collected = True
                    except:
                        time.sleep(5)

                if len(jobs) >= num_jobs:
                    break

            start_index = len(job_cards)

            try:
                show_more = driver.find_element(By.CSS_SELECTOR, ".button_Button__MlD2g.button-base_Button__knLaX[data-test='load-more']")
                driver.execute_script("arguments[0].click();", show_more)
                time.sleep(3)
            except NoSuchElementException:
                print(f"Scraping ended early. Requested: {num_jobs}, Collected: {len(jobs)}")
                break

    except Exception as e:
        print(f"Scraping interrupted due to: {e}")
    
    finally:
        driver.quit()
        return pd.DataFrame(jobs)

In [4]:
roles = ["Data Analyst", "Data Scientist", "Machine learning engineer"]
date = datetime.now().strftime("%m-%d-%Y")
for role in roles:
    df = get_jobs(role, 500, 5) 
    filename = f"{role.replace(' ', '_')}_glassdoor_jobs_{date}.xlsx"
    df.to_excel(filename, index=False)
    print(f"Scraping complete. Data saved to '{filename}'")

Progress: 0 / 500
Progress: 1 / 500
Progress: 2 / 500
Progress: 3 / 500
Progress: 4 / 500
Progress: 5 / 500
Progress: 6 / 500
Progress: 7 / 500
Progress: 8 / 500
Progress: 9 / 500
Progress: 10 / 500
Progress: 11 / 500
Progress: 12 / 500
Progress: 13 / 500
Progress: 14 / 500
Progress: 15 / 500
Progress: 16 / 500
Progress: 17 / 500
Progress: 18 / 500
Progress: 19 / 500
Progress: 20 / 500
Progress: 21 / 500
Progress: 22 / 500
Progress: 23 / 500
Progress: 24 / 500
Progress: 25 / 500
Progress: 26 / 500
Progress: 27 / 500
Progress: 28 / 500
Progress: 29 / 500
Progress: 30 / 500
Progress: 31 / 500
Progress: 32 / 500
Progress: 33 / 500
Progress: 34 / 500
Progress: 35 / 500
Progress: 36 / 500
Progress: 37 / 500
Progress: 38 / 500
Progress: 39 / 500
Progress: 40 / 500
Progress: 41 / 500
Progress: 42 / 500
Progress: 43 / 500
Progress: 44 / 500
Progress: 45 / 500
Progress: 46 / 500
Progress: 47 / 500
Progress: 48 / 500
Progress: 49 / 500
Progress: 50 / 500
Progress: 51 / 500
Progress: 52 / 500
Pro

In [5]:
interns = ["Data Analyst Intern", "Data Scientist Intern", "Machine Learning Intern"]
for intern in interns:
    df = get_jobs(intern, 70, 5) 
    filename = f"{intern.replace(' ', '_')}_glassdoor_jobs_{date}.xlsx"
    df.to_excel(filename, index=False)
    print(f"Scraping complete. Data saved to '{filename}'")

Progress: 0 / 70
Progress: 1 / 70
Progress: 2 / 70
Progress: 3 / 70
Progress: 4 / 70
Progress: 5 / 70
Progress: 6 / 70
Progress: 7 / 70
Progress: 8 / 70
Progress: 9 / 70
Progress: 10 / 70
Progress: 11 / 70
Progress: 12 / 70
Progress: 13 / 70
Progress: 14 / 70
Progress: 15 / 70
Progress: 16 / 70
Progress: 17 / 70
Progress: 18 / 70
Progress: 19 / 70
Progress: 20 / 70
Progress: 21 / 70
Progress: 22 / 70
Progress: 23 / 70
Progress: 24 / 70
Progress: 25 / 70
Progress: 26 / 70
Progress: 27 / 70
Progress: 28 / 70
Progress: 29 / 70
Progress: 30 / 70
Progress: 31 / 70
Scraping interrupted due to: name 'NoSuchElementException' is not defined
Scraping complete. Data saved to 'Data_Analyst_Intern_glassdoor_jobs_08-01-2025.xlsx'
Progress: 0 / 70
Progress: 1 / 70
Progress: 2 / 70
Progress: 3 / 70
Progress: 4 / 70
Progress: 5 / 70
Progress: 6 / 70
Progress: 7 / 70
Progress: 8 / 70
Progress: 9 / 70
Progress: 10 / 70
Progress: 11 / 70
Progress: 12 / 70
Progress: 13 / 70
Progress: 14 / 70
Progress: 15 /